# Layered hybrid ML analytic workflow_1_define behavioral feature

## 1. Define behavioral feature 

In the first step, the tracking data of animal movement generated from DeepLabCut were used to define several behavioral features, including positional feature (distance), body orientation (angle_1 and angle_2), and the snout and body center velocities. These behavioral features were then used to train ML classifiers. 



### 1.1	Applying a rolling median filter to smooth raw DeepLabCut output

The following code is for using a rolling median filter to smooth raw DeepLabCut output. 

In [4]:
import glob
import pandas as pd
import os

files = glob.glob(r"C:\Users\*.csv") # batch processing 

for file_name in files:
    df = pd.read_csv(file_name)
    df_1 = df.iloc[3: , 1:] 
    df_medfil = df_1.rolling(window=5,center=True,axis=0).median().bfill().ffill()
    os.path.splitext(file_name)
    newfilename = os.path.splitext(file_name)[0]    
    df_medfil.to_csv(newfilename + '_mf.csv')

## 1.2 define behavioral feature

### 1.2.1 Define several essential functions

In [39]:
import numpy
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

"""Demonstration of least-squares fitting of ellipses

    __author__ = "Ben Hammel, Nick Sullivan-Molina"
    __credits__ = ["Ben Hammel", "Nick Sullivan-Molina"]
    __maintainer__ = "Ben Hammel"
    __email__ = "bdhammel@gmail.com"
    __status__ = "Development"

    Requirements 
    ------------
    Python 2.X or 3.X
    numpy
    matplotlib

    References
    ----------
    (*) Halir, R., Flusser, J.: 'Numerically Stable Direct Least Squares 
        Fitting of Ellipses'
    (**) http://mathworld.wolfram.com/Ellipse.html
    (***) White, A. McHale, B. 'Faraday rotation data analysis with least-squares 
        elliptical fitting'
"""

class LSqEllipse:

    def fit(self, data):
        """Lest Squares fitting algorithm 

        Theory taken from (*)
        Solving equation Sa=lCa. with a = |a b c d f g> and a1 = |a b c> 
            a2 = |d f g>

        Args
        ----
        data (list:list:float): list of two lists containing the x and y data of the
            ellipse. of the form [[x1, x2, ..., xi],[y1, y2, ..., yi]]

        Returns
        ------
        coef (list): list of the coefficients describing an ellipse
           [a,b,c,d,f,g] corresponding to ax**2+2bxy+cy**2+2dx+2fy+g
        """
        x, y = numpy.asarray(data, dtype=float)

        #Quadratic part of design matrix [eqn. 15] from (*)
        D1 = numpy.mat(numpy.vstack([x**2, x*y, y**2])).T
        #Linear part of design matrix [eqn. 16] from (*)
        D2 = numpy.mat(numpy.vstack([x, y, numpy.ones(len(x))])).T
        
        #forming scatter matrix [eqn. 17] from (*)
        S1 = D1.T*D1
        S2 = D1.T*D2
        S3 = D2.T*D2  
        
        #Constraint matrix [eqn. 18]
        C1 = numpy.mat('0. 0. 2.; 0. -1. 0.; 2. 0. 0.')

        #Reduced scatter matrix [eqn. 29]
        M=C1.I*(S1-S2*S3.I*S2.T)

        #M*|a b c >=l|a b c >. Find eigenvalues and eigenvectors from this equation [eqn. 28]
        eval, evec = numpy.linalg.eig(M) 

        # eigenvector must meet constraint 4ac - b^2 to be valid.
        cond = 4*numpy.multiply(evec[0, :], evec[2, :]) - numpy.power(evec[1, :], 2)
        a1 = evec[:, numpy.nonzero(cond.A > 0)[1]]
        
        #|d f g> = -S3^(-1)*S2^(T)*|a b c> [eqn. 24]
        a2 = -S3.I*S2.T*a1
        
        # eigenvectors |a b c d f g> 
        self.coef = numpy.vstack([a1, a2])
        self._save_parameters()
            
    def _save_parameters(self):
        """finds the important parameters of the fitted ellipse
        
        Theory taken form http://mathworld.wolfram

        Args
        -----
        coef (list): list of the coefficients describing an ellipse
           [a,b,c,d,f,g] corresponding to ax**2+2bxy+cy**2+2dx+2fy+g

        Returns
        _______
        center (List): of the form [x0, y0]
        width (float): major axis 
        height (float): minor axis
        phi (float): rotation of major axis form the x-axis in radians 
        """

        #eigenvectors are the coefficients of an ellipse in general form
        #a*x^2 + 2*b*x*y + c*y^2 + 2*d*x + 2*f*y + g = 0 [eqn. 15) from (**) or (***)
        a = self.coef[0,0]
        b = self.coef[1,0]/2.
        c = self.coef[2,0]
        d = self.coef[3,0]/2.
        f = self.coef[4,0]/2.
        g = self.coef[5,0]
        
        #finding center of ellipse [eqn.19 and 20] from (**)
        x0 = (c*d-b*f)/(b**2.-a*c)
        y0 = (a*f-b*d)/(b**2.-a*c)
        
        #Find the semi-axes lengths [eqn. 21 and 22] from (**)
        numerator = 2*(a*f*f+c*d*d+g*b*b-2*b*d*f-a*c*g)
        denominator1 = (b*b-a*c)*( (c-a)*numpy.sqrt(1+4*b*b/((a-c)*(a-c)))-(c+a))
        denominator2 = (b*b-a*c)*( (a-c)*numpy.sqrt(1+4*b*b/((a-c)*(a-c)))-(c+a))
        width = numpy.sqrt(numerator/denominator1)
        height = numpy.sqrt(numerator/denominator2)

        # angle of counterclockwise rotation of major-axis of ellipse to x-axis [eqn. 23] from (**)
        # or [eqn. 26] from (***).
        phi = .5*numpy.arctan((2.*b)/(a-c))

        self._center = [x0, y0]
        self._width = width
        self._height = height
        self._phi = phi

    @property
    def center(self):
        return self._center

    @property
    def width(self):
        return self._width

    @property
    def height(self):
        return self._height

    @property
    def phi(self):
        """angle of counterclockwise rotation of major-axis of ellipse to x-axis 
        [eqn. 23] from (**)
        """
        return self._phi

    def parameters(self):
        return self.center, self.width, self.height, self.phi


def make_test_ellipse(center=[1,1], width=1, height=.6, phi=3.14/5):
    """Generate Elliptical data with noise
    
    Args
    ----
    center (list:float): (<x_location>, <y_location>)
    width (float): semimajor axis. Horizontal dimension of the ellipse (**)
    height (float): semiminor axis. Vertical dimension of the ellipse (**)
    phi (float:radians): tilt of the ellipse, the angle the semimajor axis
        makes with the x-axis 

    Returns
    -------
    data (list:list:float): list of two lists containing the x and y data of the
        ellipse. of the form [[x1, x2, ..., xi],[y1, y2, ..., yi]]
    """
    t = numpy.linspace(0, 2*numpy.pi, 1000)
    x_noise, y_noise = numpy.random.rand(2, len(t))
    
    ellipse_x = center[0] + width*numpy.cos(t)*numpy.cos(phi)-height*numpy.sin(t)*numpy.sin(phi) + x_noise/2.
    ellipse_y = center[1] + width*numpy.cos(t)*numpy.sin(phi)+height*numpy.sin(t)*numpy.cos(phi) + y_noise/2.

    return [ellipse_x, ellipse_y]

In [40]:
def fit_circle(points):
    
    X = np.asarray(points[::2])
    Y = np.asarray(points[1::2])
    data = [X,Y]
    lsqe = LSqEllipse()
    lsqe.fit(data)
    center, width, height, phi = lsqe.parameters()

    # Calculate the radius of Circle by getting the average of width and heigh of the Ellipse
    radii = (width + height) / 2.0
    
    return radii, center

In [41]:
def is_inside(circle_x, circle_y, rad, x, y): 
      
    # Compare radius of circle 
    # with distance of its center 
    # from given point 
    circle_x = np.asscalar(circle_x)
    circle_y = np.asscalar(circle_y)
    rad = np.asscalar(rad)
    x = float(x)
    y = float(y)
    
    if ((x - circle_x) * (x - circle_x) + 
        (y - circle_y) * (y - circle_y) <= rad * rad): 
        return True
    else: 
        return False

In [42]:
from scipy.spatial import distance

# Write the functions for caculating the angle between the vector and the x coordinate
# source: https://python-decompiler.com/article/2011-09/how-to-calculate-the-angle-between-a-line-and-the-horizontal-axis
from math import *
def angle_trunc(a):
    while a < 0.0:
        a += pi * 2
    return a

def getAngleBetweenPoints(x_orig, y_orig, x_landmark, y_landmark):
    deltaY = y_landmark - y_orig
    deltaX = x_landmark - x_orig
    return angle_trunc(atan2(deltaY, deltaX))

In [43]:
from math import *
def angle_vector_pos_x(snout_x, snout_y, v_start_x, v_start_y):
    # second method
    # http://www.mathskey.com/question2answer/28137/what-the-angle-between-given-vector-positive-direction-axis
    a_1 = snout_x - v_start_x
    a_2 = snout_y - v_start_y
    b_1 = 1
    b_2 = 0
    cos_theta = (a_1 * b_1 + a_2 * b_2) / (np.sqrt(a_1**2 + a_2**2) * np.sqrt(b_1**2 + b_2**2))
    angle = np.arccos(cos_theta)
    if a_2 >= 0:                            #quadrant 3/4
        angle = degrees(angle)
    else:    
        angle = 360 - (degrees(angle))   #direction vector with x axis
    
    v = [a_1, a_2]
    return angle,v


In [44]:
from math import * 
def process_csv(filepath,outputfile):
    import csv
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    import math
    from pandas import DataFrame
    import pandas as pd
    from scipy.spatial import distance
    row_num = 2
       
    with open(filepath) as csvfile:
        dict_data = []
        readCSV = csv.reader(csvfile, delimiter=',')
        first_row = next(readCSV)
        #second_row = next(readCSV)
        #third_row = next(readCSV)
        for row in readCSV:
            data = dict()
            points = row[1:3] + row[4:6] + row[7:9] + row[10:12] + row[13:15] + row[16:18] + row[19:21] + row[22:24]
            frame_num = row[0]
            snout = row[25:27]
            l_ear = row[28:30]
            r_ear = row[31:33]
            tail =row[34:36]
            #print(points)
            try:
                radii, center = fit_circle(points)
            except Exception as e:
                print("An exception occurred: ", row_num) 
            else:             
                v_start_x = (float(l_ear[0]) + float(r_ear[0]))/2 
                v_start_y = (float(l_ear[1]) + float(r_ear[1]))/2
                angle,v = angle_vector_pos_x(float(snout[0]), float(snout[1]), v_start_x, v_start_y)
                dist = distance.euclidean((float(snout[0]), float(snout[1])), (float(center[0].real), float(center[1].real)))
                data['frame_num'] = frame_num
                data['center_x'] = center[0]
                data['center_y'] = center[1]
                data['radii'] = radii
                data['snout_x'] = float(snout[0])
                data['snout_y'] = float(snout[1])
                data['tail_x'] = float(tail[0])
                data['tail_y'] = float(tail[1])
                data['v_x'] = v[0]
                data['v_y'] = v[1]
                data['center_ear_x']= v_start_x
                data['center_ear_y']= v_start_y
                data['angle_1'] = angle  #direction vector with x axis
                data['body_center_x'] = (v_start_x + float(tail[0]))/2 
                data['body_center_y'] = (v_start_y + float(tail[1]))/2 
                data['distance'] = dist
                dict_data.append(data)
                row_num = row_num + 1

    csv_columns = ['frame_num','center_x', 'center_y', 'radii', 'snout_x','snout_y','tail_x','tail_y', 'v_x','v_y',
               'center_ear_x', 'center_ear_y','angle_1', 'body_center_x','body_center_y','distance']
    csv_file = outputfile
        
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("I/O error")
  
  
    # angle btw direction vecter (v_1, v_2) and target vecter (c_1, c_2) from snout to center    
    df = pd.read_csv(csv_file)
    c_1 = df['center_x'] - df['snout_x'] 
    c_2 = df['center_y'] - df['snout_y'] 
    v_1 = df['v_x']
    v_2 = df['v_y']
    cos_theta = (c_1 * v_1 + c_2 * v_2) / (np.sqrt(c_1**2 + c_2**2) * np.sqrt(v_1**2 + v_2**2))
    angle2 = np.arccos(cos_theta)
    angle2 = np.degrees(angle2)  


    # https://math.stackexchange.com/questions/878785/how-to-find-an-angle-in-range0-360-between-2-vectors/879474
    # https://stackoverflow.com/questions/14066933/direct-way-of-computing-clockwise-angle-between-2-vectors/16544330#16544330
    #  the dot product is proportional to the cosine of the angle, the determinant is proprortional to its sine

    # target vetcor : c_1 and c_2 
    # dicrection vetor: v_1 and v_2
    #!!!! coordinate: x pointing right and y down
    #!!!!! (-90. 90) maybe towards petri dish

    dot_product = c_1*v_1 + c_2*v_2  # dot product

    #determinant = c_1*v_1 - c_2*v_2  # determinant
    determinant = c_1*v_2 - c_2*v_1  # determinant

    angle_3 = np.arctan2(determinant, dot_product) 
    
    angle_3 = np.degrees(angle_3)   


    df['angle_2'] = angle2
    df['angle_3'] = angle_3

    #Velocity
    #Reference: https://github.com/AlexEMG/DLCutils/blob/8a0bfacdce98ff7ba31d8d7ecaff8c6e70870e52/time_in_each_roi.py

    import numpy as np
    from scipy.spatial import distance
    import pandas as pd
    

    #velocity of snout
    df_1 = df[['frame_num', 'snout_x', 'snout_y']]
    vel_snout = []

    for i in range(len(df_1)):
        if i == 0:
            d = 0
        else:
            d = distance.euclidean(df_1.iloc[i-1, 1:3].values, df_1.iloc[i, 1:3].values)
        vel_snout.append(d)
    df['velocity_snout'] = vel_snout

    #velocity of center body
    df_2 = df[['frame_num', 'body_center_x', 'body_center_y']]
    vel_body_center = []

    for i in range(len(df_2)):
        if i == 0:
            d = 0
        else:
            d = distance.euclidean(df_2.iloc[i-1, 1:3].values, df_2.iloc[i, 1:3].values)
        vel_body_center.append(d)
    df['velocity_body_center'] = vel_body_center

    dist_dff = df['distance']-df['radii']
    df['dist_radii']= dist_dff
    df.head() 

    ### diff() Function

    distance_diff = df['distance'].diff()
    angle_1_diff = df['angle_1'].diff()
    angle_2_diff = df['angle_2'].diff()
    angle_3_diff = df['angle_3'].diff()
    snout_x_diff = df['snout_x'].diff()
    snout_y_diff = df['snout_y'].diff()
    center_ear_y_diff = df['center_ear_y'].diff()
    velocity_snout_diff = df['velocity_snout'].diff()
    velocity_body_center_diff = df['velocity_body_center'].diff()

    df['diff_distance']= distance_diff
    df['diff_angle_1']= angle_1_diff
    df['diff_angle_2']= angle_2_diff
    df['diff_angle_3']= angle_3_diff
    df['diff_snout_x']= snout_x_diff
    df['diff_snout_y']= snout_y_diff
    df['diff_center_ear_y']= center_ear_y_diff
    df['diff_velocity_snout']= velocity_snout_diff
    df['diff_velocity_body_center']= velocity_body_center_diff

    df.to_csv(outputfile)

   
    return df
    

### 1.3.2 Batch process of defining behavioral feature

In [3]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from pandas import DataFrame
import pandas as pd
from math import *   
from scipy.spatial import distance
import pandas as pd
import glob
import os


row_num = 2

csvfiles = glob.glob(r"C:\Users\*.csv") # batch processing
output_num = 1

for csvfile in csvfiles:
    os.path.splitext(csvfile)
    newfilename = os.path.splitext(csvfile)[0]    
    outputfile = newfilename + '_featured.csv'
    
    print(csvfile)
    process_csv(csvfile, outputfile)
    output_num = output_num + 1

